In [1]:
import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import psycopg2


### Data and Functions for Optimization

In [26]:
con = psycopg2.connect(database="postgres", user="postgres", password="0225", host="localhost", port="5432")

query = \
'''
select 
    id, year, week, type_2, window_size, window_start, window_end, promo_units, promo_weight as original_weights, window_sum_lifted_estimated_units_raw as sum_lifted_raw, window_frame_reg_units_1, window_frame_reg_units_2, estimated_reg_units_2
    
from sales_7
where 
    up_degree_2 >= 1 
and window_size >= 3
order by id, year, week

'''

data = pd.read_sql(query, con=con)
data.head()

,id,year,week,type_2,window_size,window_start,window_end,promo_units,original_weights,sum_lifted_raw,window_frame_reg_units_1,window_frame_reg_units_2,estimated_reg_units_2
0,0005,2018,5,P,3,1,0,10465,0.272,14771.35961,3046,4396,4017.809814
1,0005,2018,6,P,3,0,0,17762,0.462,14771.35961,3046,4396,6824.368140
2,0005,2018,7,P,3,0,1,10181,0.265,14771.35961,3046,4396,3914.410297
3,0015,2017,41,P,3,1,0,9623,0.376,15104.54084,4833,3209,5679.307356
4,0015,2017,42,P,3,0,0,7320,0.286,15104.54084,4833,3209,4319.898680


In [24]:
len(data)

3679

In [25]:
data = data.astype({'year': str, 'week': str})

In [5]:
# zscore
scaler = StandardScaler()
# scaler.fit(data['promo_units']) # error
scaler.fit(np.expand_dims(data['promo_units'], axis=1))
transformed_promo_units_z = scaler.transform(np.expand_dims(data['promo_units'], axis=1))
transformed_promo_units_z = np.squeeze(transformed_promo_units_z, axis=1)

In [ ]:
# wrap it within a function
def zscore_normalization(series): # data range [negative, positive]
    scaler = StandardScaler()
    scaler.fit(np.expand_dims(series, axis=1))
    transformed_promo_units = scaler.transform(np.expand_dims(series, axis=1))
    transformed_promo_units = np.squeeze(transformed_promo_units, axis=1)
    return transformed_promo_units

In [9]:
factor = 2

data['nor_promo_units'] = data.groupby(by='id')['promo_units'].transform(zscore_normalization)  # calculate NORMALIZED_D_SUM
data['factored_promo_units'] = data['nor_promo_units'].map(lambda x: math.exp(x/factor)) # KK, same as .transform above. Compare and test execution speed, also with list(map...)

In [10]:
data.head()

,id,year,week,type_2,window_size,window_start,window_end,promo_units,sum_lifted_raw,window_frame_reg_units_1,window_frame_reg_units_2,nor_promo_units,factored_promo_units
0,0005,2018,5,P,3,1,0,10465,14771.35961,3046,4396,-0.666250,0.716681
1,0005,2018,6,P,3,0,0,17762,14771.35961,3046,4396,1.413441,2.027332
2,0005,2018,7,P,3,0,1,10181,14771.35961,3046,4396,-0.747192,0.688255
3,0015,2017,41,P,3,1,0,9623,15104.54084,4833,3209,1.165278,1.790758
4,0015,2017,42,P,3,0,0,7320,15104.54084,4833,3209,-1.276610,0.528187


In [11]:
# demo
# a = pd.Series([2,2,2,2])
# a / a.sum()

In [13]:
data['weights'] = data.groupby(by='id')['factored_promo_units'].map(lambda x: x / x.sum())
data['factored_bl'] = data['sum_lifted_raw'] * data['weights']

In [14]:
data.head()

,id,year,week,type_2,window_size,window_start,window_end,promo_units,sum_lifted_raw,window_frame_reg_units_1,window_frame_reg_units_2,nor_promo_units,factored_promo_units,weights,factored_bl
0,0005,2018,5,P,3,1,0,10465,14771.35961,3046,4396,-0.666250,0.716681,0.208807,3084.359727
1,0005,2018,6,P,3,0,0,17762,14771.35961,3046,4396,1.413441,2.027332,0.590668,8724.974761
2,0005,2018,7,P,3,0,1,10181,14771.35961,3046,4396,-0.747192,0.688255,0.200525,2962.025122
3,0015,2017,41,P,3,1,0,9623,15104.54084,4833,3209,1.165278,1.790758,0.530408,8011.568887
4,0015,2017,42,P,3,0,0,7320,15104.54084,4833,3209,-1.276610,0.528187,0.156445,2363.025464


In [15]:
def cal_loss(row):
    if row['window_start'] == 1:
        row['loss'] = row['factored_bl'] - row['window_frame_reg_units_1']
    if row['window_end'] == 1:
        row['loss'] = row['factored_bl'] - row['window_frame_reg_units_2']
    
    return row

In [17]:
data = data.apply(cal_loss, axis=1)
data.head()

,factored_bl,factored_promo_units,id,loss,nor_promo_units,promo_units,sum_lifted_raw,type_2,week,weights,window_end,window_frame_reg_units_1,window_frame_reg_units_2,window_size,window_start,year
0,3084.359727,0.716681,0005,38.359727,-0.666250,10465,14771.35961,P,5,0.208807,0,3046,4396,3,1,2018
1,8724.974761,2.027332,0005,NaN,1.413441,17762,14771.35961,P,6,0.590668,0,3046,4396,3,0,2018
2,2962.025122,0.688255,0005,-83.974878,-0.747192,10181,14771.35961,P,7,0.200525,1,3046,4396,3,0,2018
3,8011.568887,1.790758,0015,3178.568887,1.165278,9623,15104.54084,P,41,0.530408,0,4833,3209,3,1,2017
4,2363.025464,0.528187,0015,NaN,-1.276610,7320,15104.54084,P,42,0.156445,0,4833,3209,3,0,2017


In [19]:
# data['loss2'] = data['window_start'] * (data['factored_bl'] - data['window_start'] * data['window_frame_reg_units_1']) + data['window_end'] * (data['factored_bl'] - data['window_end'] * data['window_frame_reg_units_2'])

# Calculating loss
loss = - data['loss'].sum()

In [8]:
def zscore_normalization(series): # data range [negative, positive]
    scaler = StandardScaler()
    scaler.fit(np.expand_dims(series, axis=1))
    transformed_promo_units = scaler.transform(np.expand_dims(series, axis=1))
    transformed_promo_units = np.squeeze(transformed_promo_units, axis=1)
    return transformed_promo_units

In [6]:
def minmax_normalization(series): # data range (0,1)
    scaler = MinMaxScaler()
    scaler.fit(np.expand_dims(series, axis=1))
    transformed_promo_units = scaler.transform(np.expand_dims(series, axis=1))
    transformed_promo_units = np.squeeze(transformed_promo_units, axis=1)
    return transformed_promo_units

In [7]:
def get_weights(series):
    return series / series.sum()

In [8]:
def get_loss(window_start, window_end, factored_bl, reg_units_1, reg_units_2):
    loss = window_start * (factored_bl - window_start * reg_units_1) + window_end * (factored_bl - window_end * reg_units_2)

    return loss

In [10]:
def zscore_target(factor, df_inp = data):
    data['nor_promo_units'] = data.groupby(by='id')['promo_units'].transform(zscore_normalization)  
    data['factored_promo_units'] = data['nor_promo_units'].map(lambda x: math.exp(x/factor)) 
    data['weights'] = data.groupby(by='id')['factored_promo_units'].map(lambda x: x / x.sum())
    data['factored_bl'] = data['sum_lifted_raw'] * data['weights']
    data['loss'] = get_loss(data['window_start'], data['window_end'], data['factored_bl'], data['window_frame_reg_units_1'], data['window_frame_reg_units_2'])
    data['abs_loss'] = list(map(abs, data['loss']))
    loss = df_inp['abs_loss'].sum() / len(data)
    
    return -loss

In [9]:
def minmax_target(factor, df_inp = data):
    data['nor_promo_units'] = data.groupby(by='id')['promo_units'].transform(minmax_normalization)  
    data['factored_promo_units'] = data['nor_promo_units'].map(lambda x: math.exp(x/factor)) 
    data['weights'] = data.groupby(by='id')['factored_promo_units'].map(lambda x: x / x.sum())
    data['factored_bl'] = data['sum_lifted_raw'] * data['weights']
    data['loss'] = get_loss(data['window_start'], data['window_end'], data['factored_bl'], data['window_frame_reg_units_1'], data['window_frame_reg_units_2'])
    data['abs_loss'] = list(map(abs, data['loss']))
    loss = df_inp['abs_loss'].sum() / len(data)
    
    return -loss

### Loss of Current Naive Method

In [11]:
# show what's the loss without any change
data_copy = data.copy()
# Calculating loss
data_copy['loss'] = get_loss(data['window_start'], data['window_end'], data['estimated_reg_units_2'], data['window_frame_reg_units_1'], data['window_frame_reg_units_2'])
v1_loss = data_copy['loss'].sum()
print('original loss is {}, see if optimization could reduce it'.format(v1_loss))

original loss is 681.5519251066904, see if optimization could reduce it


### Run Optimizer to see if new distribution could beat baseline

In [ ]:
bo_minmax = BayesianOptimization(minmax_target, {'factor': (0.2, 10)})
bo_minmax.maximize(init_points=1, n_iter=80, acq='ucb', kappa=5)

Initialization
-----------------------------------------
 Step |   Time |      Value |    factor | 
    1 | 00m23s | -735.40987 |    5.6017 | 
Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |    factor | 
    2 | 00m26s | -3119.19724 |    0.2000 | 
    3 | 00m25s | -757.11416 |   10.0000 | 
    4 | 00m26s | -744.50932 |    7.0115 | 
    5 | 00m26s | -739.21054 |    6.1402 | 
    6 | 00m29s | -736.46625 |    5.7457 | 


C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


    7 | 00m27s | -735.06129 |    5.5553 | 


C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\App

C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\App

...

C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\App

C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\App

C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\App

C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
C:\Users\oliver.song\App

   28 | 00m28s | -719.82395 |    3.8396 | 


In [ ]:
bo_zscore = BayesianOptimization(zscore_target, {'factor': (0.2, 10)})
bo_zscore.maximize(init_points=1, n_iter=80, acq='ucb', kappa=5)